In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Loading of food price
df = pd.read_csv('https://data.humdata.org/dataset/42db041f-7aaf-4ab4-961f-2a12096861e7/resource/12b51155-0cd3-4806-9924-61ede4077591/download/wfp_food_prices_nga.csv')
df.head()

,date,admin1,admin2,market,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice
0,#date,#adm1+name,#adm2+name,#loc+market+name,#geo+lat,#geo+lon,#item+type,#item+name,#item+unit,#item+price+flag,#item+price+type,#currency,#value,#value+usd
1,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Maize,KG,actual,Wholesale,NGN,175.92,1.5525
2,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Millet,KG,actual,Wholesale,NGN,150.18,1.3254
3,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Rice (imported),KG,actual,Wholesale,NGN,358.7,3.1656
4,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Sorghum,KG,actual,Wholesale,NGN,155.61,1.3733


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78312 entries, 0 to 78311
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       78312 non-null  object
 1   admin1     78312 non-null  object
 2   admin2     78312 non-null  object
 3   market     78312 non-null  object
 4   latitude   78312 non-null  object
 5   longitude  78312 non-null  object
 6   category   78312 non-null  object
 7   commodity  78312 non-null  object
 8   unit       78312 non-null  object
 9   priceflag  78312 non-null  object
 10  pricetype  78312 non-null  object
 11  currency   78312 non-null  object
 12  price      78312 non-null  object
 13  usdprice   78312 non-null  object
dtypes: object(14)
memory usage: 8.4+ MB


In [4]:
 #check missing values
missing_data = df.isna().sum()
missing_data

date         0
admin1       0
admin2       0
market       0
latitude     0
longitude    0
category     0
commodity    0
unit         0
priceflag    0
pricetype    0
currency     0
price        0
usdprice     0
dtype: int64

In [5]:
df.shape #shape of the data (rows, columns)

(78312, 14)

In [6]:
df.duplicated().sum() #check for duplicate values

0

In [7]:
# Rename columns
df.rename(columns={'admin1': 'State', 'admin2': 'Local_Government', 'market': 'Market_Name'}, inplace=True)

df = df.rename(columns={'admin2' : 'local_gov_area'})

# Drop the first row
df = df.iloc[1:]

# Convert the 'Date' column to datetime type
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

# Create a mapping dictionary for month names
month_mapping = {1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June',
                 7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'}

# Use the map function to replace the values in the 'month' column
df['month'] = df['month'].map(month_mapping)

# Drop rows where priceflag is "forecast"
df = df[df['priceflag'] != 'forecast']

# Reset the index
df.reset_index(drop=True, inplace=True)

# Display the updated DataFrame
df.head()

,date,State,Local_Government,Market_Name,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice,year,month
0,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Maize,KG,actual,Wholesale,NGN,175.92,1.5525,2002,January
1,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Millet,KG,actual,Wholesale,NGN,150.18,1.3254,2002,January
2,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Rice (imported),KG,actual,Wholesale,NGN,358.7,3.1656,2002,January
3,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Sorghum,KG,actual,Wholesale,NGN,155.61,1.3733,2002,January
4,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,pulses and nuts,Beans (niebe),KG,actual,Wholesale,NGN,196.87,1.7374,2002,January


In [8]:
df.shape

(64221, 16)

In [9]:
def convert_to_kg(row):
    unit = row['unit']
    price = row['price']

    # Values to exclude from conversion
    exclude_values = ['30 pcs', '100 Tubers']

    # Check if the unit is not in the exclude list
    if unit not in exclude_values:
        # Check if the unit contains 'KG' and has a numeric prefix
        if 'KG' in unit.upper() and any(char.isdigit() for char in unit):
            # Extract the numeric value from the unit
            numeric_value = float(''.join(filter(str.isdigit, unit)))

            # Convert the price to a numerical value
            price = float(price)

            # Calculate price per 1 KG
            if numeric_value > 0:
                price_per_kg = price / numeric_value

                # Update the unit to '1 KG'
                unit = '1KG'

                # Return the updated values
                return price_per_kg, unit

    # If the unit is not eligible for conversion, return the original values
    return price, unit

# Apply the function to calculate price per 1 KG and update the unit and usd_price columns
df[['price', 'unit']] = df.apply(convert_to_kg, axis=1, result_type='expand')

# Replace "KG" with "1 KG" while keeping "1 KG" unchanged
df['unit'] = df['unit'].apply(lambda x: '1KG' if x == 'KG' else x)

df.head()

,date,State,Local_Government,Market_Name,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice,year,month
0,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Maize,1KG,actual,Wholesale,NGN,175.92,1.5525,2002,January
1,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Millet,1KG,actual,Wholesale,NGN,150.18,1.3254,2002,January
2,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Rice (imported),1KG,actual,Wholesale,NGN,358.7,3.1656,2002,January
3,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Sorghum,1KG,actual,Wholesale,NGN,155.61,1.3733,2002,January
4,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,pulses and nuts,Beans (niebe),1KG,actual,Wholesale,NGN,196.87,1.7374,2002,January


In [10]:
def convert_to_kg(row):
    unit = row['unit']
    price = row['price']

    # Check if the unit is one of the specified units to convert
    if unit in ['100 L', '20 G', '750 ML', '400 G', '300 G', '250 G']:
        # Convert the 'price' column to a numerical data type
        price = pd.to_numeric(price)

        # Calculate the equivalent price for 1 KG
        if unit == '100 L':
            price_per_kg = price / 100  # Convert 100 L to 1 KG
        elif unit == '20 G':
            price_per_kg = price * 50   # Convert 20 G to 1 KG
        elif unit == '750 ML':
            price_per_kg = price * 1.3333  # Convert 750 ML to 1 KG (approximately)
        elif unit == '400 G':
            price_per_kg = price * 2.5  # Convert 400 G to 1 KG
        elif unit == '300 G':
            price_per_kg = price * 3.3333  # Convert 300 G to 1 KG (approximately)
        elif unit == '250 G':
            price_per_kg = price * 4.0  # Convert 250 G to 1 KG

        unit = '1KG'  # Update the unit to '1 KG'

        # Return as a tuple
        return price_per_kg, unit

    # If the unit is not in the specified units, return the original price and unit
    return price, unit

# Apply the function to calculate price per 1 KG and update the unit
df[['price', 'unit']] = df.apply(convert_to_kg, axis=1, result_type='expand')
df.head()


,date,State,Local_Government,Market_Name,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice,year,month
0,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Maize,1KG,actual,Wholesale,NGN,175.92,1.5525,2002,January
1,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Millet,1KG,actual,Wholesale,NGN,150.18,1.3254,2002,January
2,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Rice (imported),1KG,actual,Wholesale,NGN,358.7,3.1656,2002,January
3,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Sorghum,1KG,actual,Wholesale,NGN,155.61,1.3733,2002,January
4,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,pulses and nuts,Beans (niebe),1KG,actual,Wholesale,NGN,196.87,1.7374,2002,January


In [11]:
# Define a function to map months to seasons
def map_to_season(month):
    if month in ['December', 'January', 'February']:
        return 'Dry Season'
    elif month in ['March', 'April', 'May']:
        return 'Early Rainy Season'
    elif month in ['June', 'July', 'August', 'September']:
        return 'Peak Rainy Season'
    else:
        return 'Late Rainy Season'

# Apply the function to create the 'season' column
df['season'] = df['month'].apply(map_to_season)
df.head()


,date,State,Local_Government,Market_Name,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice,year,month,season
0,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Maize,1KG,actual,Wholesale,NGN,175.92,1.5525,2002,January,Dry Season
1,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Millet,1KG,actual,Wholesale,NGN,150.18,1.3254,2002,January,Dry Season
2,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Rice (imported),1KG,actual,Wholesale,NGN,358.7,3.1656,2002,January,Dry Season
3,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Sorghum,1KG,actual,Wholesale,NGN,155.61,1.3733,2002,January,Dry Season
4,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,pulses and nuts,Beans (niebe),1KG,actual,Wholesale,NGN,196.87,1.7374,2002,January,Dry Season


In [12]:
df['unit'].unique()

array(['1KG', 'Unit', 'L', '30 pcs', '100 Tubers'], dtype=object)

In [13]:
df.columns

Index(['date', 'State', 'Local_Government', 'Market_Name', 'latitude',
       'longitude', 'category', 'commodity', 'unit', 'priceflag', 'pricetype',
       'currency', 'price', 'usdprice', 'year', 'month', 'season'],
      dtype='object')

In [14]:
# Loading of Inflation price
df_inflation = pd.read_csv("https://dagshub.com/Omdena/KanoNigeriaChapter_FoodPrices/raw/278dd0522a02d7e1407777bb82a1f796b399b3ed/task%20%20/task%201_%20data%20collection%20%20and%20preprocessing%20/inflation_data.csv")
df_inflation.head()

,DATE,Inflation
0,01-01-1960,5.444327
1,01-01-1961,6.279147
2,01-01-1962,5.265632
3,01-01-1963,-2.694655
4,01-01-1964,0.856793


In [15]:
df_inflation['DATE'] = pd.to_datetime(df_inflation['DATE'], format='%d-%m-%Y')
df_inflation['year'] = df_inflation['DATE'].dt.year

# Filter data from 2002 to the last year in the DataFrame
df_inf_filter= df_inflation[df_inflation['year'] >= 2002]

df_inf_filter.drop(columns=['DATE'], inplace=True)
df_inf_filter.head()

,Inflation,year
42,12.876579,2002
43,14.031784,2003
44,14.998034,2004
45,17.863493,2005
46,8.225222,2006


In [16]:
merged_df = df.merge(df_inf_filter, on=['year'], how='left')
merged_df.head()


,date,State,Local_Government,Market_Name,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice,year,month,season,Inflation
0,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Maize,1KG,actual,Wholesale,NGN,175.92,1.5525,2002,January,Dry Season,12.876579
1,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Millet,1KG,actual,Wholesale,NGN,150.18,1.3254,2002,January,Dry Season,12.876579
2,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Rice (imported),1KG,actual,Wholesale,NGN,358.7,3.1656,2002,January,Dry Season,12.876579
3,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Sorghum,1KG,actual,Wholesale,NGN,155.61,1.3733,2002,January,Dry Season,12.876579
4,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,pulses and nuts,Beans (niebe),1KG,actual,Wholesale,NGN,196.87,1.7374,2002,January,Dry Season,12.876579


In [17]:
merged_df.columns

Index(['date', 'State', 'Local_Government', 'Market_Name', 'latitude',
       'longitude', 'category', 'commodity', 'unit', 'priceflag', 'pricetype',
       'currency', 'price', 'usdprice', 'year', 'month', 'season',
       'Inflation'],
      dtype='object')

In [18]:
merged_df.shape

(64221, 18)

In [19]:
# Save the cleaned DataFrame to a new CSV file
#merged_df.to_csv('preprocessed_data.csv', index=False)

In [20]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64221 entries, 0 to 64220
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              64221 non-null  datetime64[ns]
 1   State             64221 non-null  object        
 2   Local_Government  64221 non-null  object        
 3   Market_Name       64221 non-null  object        
 4   latitude          64221 non-null  object        
 5   longitude         64221 non-null  object        
 6   category          64221 non-null  object        
 7   commodity         64221 non-null  object        
 8   unit              64221 non-null  object        
 9   priceflag         64221 non-null  object        
 10  pricetype         64221 non-null  object        
 11  currency          64221 non-null  object        
 12  price             64221 non-null  object        
 13  usdprice          64221 non-null  object        
 14  year              6422

In [21]:
# converts the specified columns to their appropriate data types
merged_df['latitude'] = merged_df['latitude'].astype(float)
merged_df['longitude'] = merged_df['longitude'].astype(float)
merged_df['price'] = merged_df['price'].astype(float)
merged_df['usdprice'] = merged_df['usdprice'].astype(float)
merged_df['Inflation'] = merged_df['Inflation'].astype(float)
merged_df['date'] = pd.to_datetime(merged_df['date'])


In [22]:
merged_df.corr()

,latitude,longitude,price,usdprice,year,Inflation
latitude,1.000000,0.434396,-0.025368,-0.036598,-0.076376,-0.029596
longitude,0.434396,1.000000,-0.004234,-0.045452,0.154937,0.061844
price,-0.025368,-0.004234,1.000000,0.023319,0.077559,0.082633
usdprice,-0.036598,-0.045452,0.023319,1.000000,-0.033059,0.035062
year,-0.076376,0.154937,0.077559,-0.033059,1.000000,0.529896
Inflation,-0.029596,0.061844,0.082633,0.035062,0.529896,1.000000


In [23]:
# Save the cleaned DataFrame to a new CSV file
merged_df.to_csv('preprocessed-data.csv', index=False)